<a href="https://colab.research.google.com/github/annariha/StanCon-2024-BO-Stan/blob/main/template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bayesian optimisation using Stan @ StanCon 2024

### Setup

In [ ]:
# Use a repository of pre-built package binaries to speed-up installation
download.file("https://github.com/eddelbuettel/r2u/raw/master/inst/scripts/add_cranapt_jammy.sh",
              "add_cranapt_jammy.sh")
Sys.chmod("add_cranapt_jammy.sh", "0755")
system("./add_cranapt_jammy.sh")

# Install R Packages
install.packages(c("here", "tidyverse", "ggplot2", "MASS", "bayesplot", "cmdstanr"),
                  repos = c("https://stan-dev.r-universe.dev", getOption("repos")))


In this tutorial, we use the [cmdstanr](https://mc-stan.org/cmdstanr/articles/cmdstanr.html) R interface to CmdStan. We install and setup CmdStan as follows:

In [ ]:
# Install and setup CmdStan
download.file("https://github.com/stan-dev/cmdstan/releases/download/v2.35.0/colab-cmdstan-2.35.0.tgz",
              "cmdstan-2.35.0.tgz")
utils::untar("cmdstan-2.35.0.tgz")
cmdstanr::set_cmdstan_path("cmdstan-2.35.0")

Now, we load all required libraries and set a seed: 

In [ ]:
library(here)
library(tidyverse)
library(ggplot2)
library(MASS)
library(cmdstanr)
library(khroma)

set.seed(424242)

### 1. Icebreaker 

Imagine you only observe three function evaluations of an otherwise unknown function, and you want to find the global minimum of the function. 

How would you approach this? 

In [ ]:
observed_evals <- data.frame(x = c(0.2, 0.25, 0.8), y = c(0, -0.2, 5))

ggplot(data = observed_evals, aes(x = x, y = y)) +
    geom_point() + 
    ylab("f(x)") +
    ylim(-5, 5) +
    theme_bw()

### 2. Introduction to Bayesian optimisation (BO)

Luckily, we don't have to continue guessing what points to evaluate next. Bayesian optimisation to the rescue! 

The goal of BO is to find the minimum or maximum of an unknown function for which we can only obtain function evaluations at a finite number of points. 

There are many applications of BO ranging from hyperparameter discovery to experimental design. 

The BO mechanism makes use of a **surrogate model** and an **acquisition function** to efficiently navigate the trade-off between exploration and exploitation. 

### 3. Surrogate models

One option for a surrogate model is a Gaussian process.  

By definition of a GP, writing $g \sim \mathcal{GP}\left(\mu, K\right)$ with mean function $\mu(.)$ and a covariance or kernel function $K(.,.)$ means that the joint distribution of the function’s value $g(\mathbf{x})$ at a finite number of input points $\mathbf{x} = \{x_1, \cdots, x_n\}$ is a multivariate normal distribution.

In [ ]:
# TODO: Hands-on Exercise: implement your own kernel in Stan  
# TODO: add an example for inference from GP kernel function & visualisation

# squared exponential kernel 

# rational quadratic kernel 

To illustrate how to setup the different components of Bayesian optimisation, assume that the unknown function is $f(x) = (6  x - 2)^2  \sin(12  x - 4)$ (aka Forrester function). We can start with the following GP surrogate for $f(x)$:  

$$
\begin{aligned}
y &\sim \text{N}(g(x), \sigma) \ \text{with} \ \sigma \sim \text{N}^+(0,1),\\
\\
g(x) &\sim GP(\mu, K),  \text{with} \ \mu \sim \text{N}(0,1),\\ 
K_{i,j} &= k (x_i, x_j) = \alpha^2  \exp \left(- \frac{(x_i - x_j)^2}{\rho^2} \right),\\
\\
\alpha &\sim \text{N}^+(0,1),\\ 
\rho &\sim \text{N}(0.3,0.1).
\end{aligned}
$$

In Stan, we can implement the model like this: 

In [ ]:
stan_program <- "
data {
  int<lower=1> N_obs;
  array[N_obs] real x_obs;
  vector[N_obs] y_obs;
  int<lower=1> N_pred;
  array[N_pred] real x_pred;
}

transformed data {
  int<lower=1> N = N_obs + N_pred;
  array[N] real x;
  for (n_obs in 1:N_obs)   x[n_obs] = x_obs[n_obs];
  for (n_pred in 1:N_pred) x[N_obs + n_pred] = x_pred[n_pred];
}

parameters {
  real<lower=0> rho;
  real<lower=0> alpha;
  real<lower=0> sigma;
  real mu;
  vector[N] eta;
}

transformed parameters{
  vector[N] g;
  { 
    matrix[N, N] L;
    matrix[N, N] K;
    K = gp_exp_quad_cov(x, alpha, rho) + diag_matrix(rep_vector(1e-10, N));
    L = cholesky_decompose(K);
    g = mu + L * eta;
  }
}

model {
  rho   ~ normal(0.3,0.1);
  alpha ~ std_normal();
  sigma ~ std_normal();
  mu    ~ std_normal();
  eta   ~ std_normal();
  y_obs ~ normal(g[1:N_obs], sigma);
}

generated quantities {
  vector[N_pred] y_pred;
  for (n_pred in 1:N_pred){
    y_pred[n_pred] = normal_rng(g[N_obs + n_pred], sigma);
  }
}

"

Note that this one implementation to get started, and there are other (more efficient) ways to set up GPs in Stan, see the comments by Andrew Johnson and Aki Vehtari in [Stan discourse]() and Aki's case study on GPs [here](). 

In [ ]:
# Save Stan code in Stan file 
fit_gp <- cmdstanr::write_stan_file(stan_program, dir = "Stan/", basename = "fit_gp.stan")

# To print Stan code, run: cat(readLines(fit_gp, sep = "\n"))

# To use the model, we first need to compile it
model_fit_gp <- cmdstanr::cmdstan_model(fit_gp)

### 4. Acquisition functions
-> if I create a figure, put the code in the template such that participants can try

#### For example: Lower confidence bound 

$$\text{AF}_\text{LCB}(x) = \mu(x) - \kappa*\sigma(x)$$

In [ ]:
lower_bound_acqs_fun <- function(m, s, kappa=2){
  lower_bound <- m - kappa * s
  return(lower_bound)
}

### (?) Computational Tricks for GPs 

- mention GP tricks: Kronecker, HSGP
- Thompson sampling

### 5. Cost- and response propensity-aware BO 

#### Varying cost of queries  

#### Propensity of response 

-> possibly provide samples via GitHub & then ask to visualise 
-> Aalto file sharing 

In [ ]:
true_f <- function(x) 
x_grid <- seq(0, 1, length.out = 100)
f_evals <- true_f(x_grid)
data_plot <- data.frame(x_grid, f_evals)

# Plot the objective function 
plot <- ggplot(data = data_plot, aes(x = x_grid, y = f_evals)) +
  geom_line() +
  labs(y = "f(x)") +
  theme_bw()

plot

Before we have collected any data, we can sample from our priors and check the predictions we would obtain. 

In [ ]:
# Get samples using chosen priors 

model_sim <- cmdstanr::cmdstan_model(stan_file = here::here("Stan", "sim_gauss.stan"))

n_draws <- 15
samples <- matrix(NA, nrow = n_draws, ncol=length(x_grid))

for (i in 1:n_draws){
    # 1. create data input, sample from the chosen priors for fit_gauss_3.stan
    stan_dat <- list(N = length(x_grid),
                     x = x_grid,
                     alpha = abs(rnorm(1)),
                     rho = rnorm(1, 0.3, 0.1),
                     mu = rnorm(1),
                     sigma = abs(rnorm(1, 0.1, 1)))

    # 2. sample from model_sim using one chain and iteration, no warmup
    gp_priors <- model_sim$sample(data = stan_dat,
                                  seed = 424242, 
                                  iter_sampling = 1, 
                                  iter_warmup = 0, 
                                  chains = 1,
                                  adapt_engaged=FALSE) # to sample without warmup
    # 3. extract corresponding samples
    samples[i,] <- gp_priors$draws("g") 
  }

In [ ]:
# Visualise the prior predictive results 


# Extract and reformat samples
data <- data.frame(samples[,1:NCOL(samples)])
colnames(data) <- sub("^X", "", colnames(data))

data <- data |>
  mutate(draw_id = as.factor(row_number())) |>
  pivot_longer(cols = -draw_id, names_to = "n_evals", values_to = "evaluations") |>
  mutate(n_evals = as.integer(n_evals)) |>
  nest(data = c(draw_id, evaluations)) |>
  mutate(x_grid = x_grid) |>
  unnest(cols = c(data))

# Plot prior predictives 
plot_prior_pred <- plot +
  geom_line(data = data, aes(x = x_grid, y = evaluations, group = draw_id, color = draw_id, alpha = 0.4)) + 
  scale_colour_discreterainbow() + 
  theme(legend.position = "none")

plot_prior_pred

### 6. Wrap-up 

- query from us function evaluations
- find the minimum of a function, but this time use the science you’ve learnt